Kivihya Aligula	136827
Joy Mithamo	136587
Adrian Oduma	127156
Macharia Wachira137492
Lesilau Eric	136712
Ali Chilumo JR 	135354
Luwate Inda	137184

In [6]:
def compute_first_sets(cfg):
    first_sets = {non_term: set() for non_term in cfg.keys()}
    terminals = set()

    # Find terminals
    for productions in cfg.values():
        for production in productions:
            for symbol in production:
                if symbol not in cfg:
                    terminals.add(symbol)

    changed = True
    while changed:
        changed = False
        for non_terminal, productions in cfg.items():
            for production in productions:
                first_set_alpha = compute_first(production, cfg)
                if first_sets[non_terminal].update(first_set_alpha):
                    changed = True

    return first_sets

def compute_follow_sets(cfg, start_symbol):
    follow_sets = {non_term: set() for non_term in cfg.keys()}
    follow_sets[start_symbol].add('$')  # Rule 1: For the start symbol, place $ in FOLLOW(S)

    def find_follow(non_terminal):
        if non_terminal in follow_sets:
            return follow_sets[non_terminal]

        follow_set = set()

        for prod_non_terminal, productions in cfg.items():
            for production in productions:
                for i, symbol in enumerate(production):
                    if symbol == non_terminal:
                        rest = production[i + 1:]

                        if not rest:
                            # Rule 2: If A → α B, then FOLLOW(B) = FOLLOW(A)
                            if follow_sets[prod_non_terminal].update(find_follow(prod_non_terminal)):
                                follow_set.update(find_follow(prod_non_terminal))
                        else:
                            # Rule 3: If A → α B β
                            first_set_beta = compute_first(rest, cfg)
                            if 'epsilon' not in first_set_beta:
                                # If ε not in FIRST(β), FOLLOW(B) = FIRST(β)
                                if follow_sets[prod_non_terminal].update(first_set_beta):
                                    follow_set.update(first_set_beta)
                            else:
                                # If ε in FIRST(β), FOLLOW(B) = (FIRST(β) - {ε}) U FOLLOW(A)
                                if follow_sets[prod_non_terminal].update(first_set_beta - {'epsilon'}) or follow_sets[prod_non_terminal].update(find_follow(prod_non_terminal)):
                                    follow_set.update(first_set_beta)

        return follow_set

    changed = True
    while changed:
        changed = False
        for non_terminal, productions in cfg.items():
            for production in productions:
                for i, symbol in enumerate(production):
                    if symbol in cfg:
                        rest = production[i + 1:]

                        if not rest:
                            # Rule 2: If A → α B, then FOLLOW(B) = FOLLOW(A)
                            if follow_sets[symbol].update(follow_sets[non_terminal]):
                                changed = True
                        else:
                            # Rule 3: If A → α B β
                            first_set_beta = compute_first(rest, cfg)
                            if 'epsilon' not in first_set_beta:
                                # If ε not in FIRST(β), FOLLOW(B) = FIRST(β)
                                if follow_sets[symbol].update(first_set_beta):
                                    changed = True
                            else:
                                # If ε in FIRST(β), FOLLOW(B) = (FIRST(β) - {ε}) U FOLLOW(A)
                                if follow_sets[symbol].update(first_set_beta - {'epsilon'}) or follow_sets[symbol].update(find_follow(non_terminal)):
                                    changed = True

    return follow_sets
#Gets the first of any given non-terminal
def compute_first(symbols, cfg):
    result = set()

    def find_first(non_terminal):
        if non_terminal in result:
            return result

        for production in cfg.get(non_terminal, []):
            for symbol in production:
                if symbol in cfg:
                    find_first(symbol)
                    if 'epsilon' not in result:
                        break
                else:
                    result.add(symbol)
                    if 'epsilon' not in result:
                        break

    for symbol in symbols:
        if symbol in cfg:
            find_first(symbol)
            if 'epsilon' not in result:
                break
        else:
            result.add(symbol)
            if 'epsilon' not in result:
                break

    return result

# Updated CFG with left recursion
example_cfg_with_recursion = {
      'S': [['A', 'b'], ['c']],
        'A': [['a', 'A'], ['#']]
}

first_sets = compute_first_sets(example_cfg_with_recursion)
follow_sets = compute_follow_sets(example_cfg_with_recursion, 'S')

# Print the computed FIRST sets
print("First Sets:")
for non_terminal, first_set in first_sets.items():
    print(f"{non_terminal}: {first_set}")

# Print the computed FOLLOW sets
print("\nFollow Sets:")
for non_terminal, follow_set in follow_sets.items():
    print(f"{non_terminal}: {follow_set}")

First Sets:
S: {'c', '#', 'a'}
A: {'#', 'a'}

Follow Sets:
S: {'$'}
A: {'b'}


In [7]:
def construct_parse_table(cfg, first_sets, follow_sets):
    parse_table = {}

    for non_terminal, productions in cfg.items():
        for production in productions:
            first_set_alpha = compute_first(production, cfg)

            for symbol in first_set_alpha:
                if symbol != 'epsilon':
                    parse_table[(non_terminal, symbol)] = production

            if 'epsilon' in first_set_alpha:
                follow_set_A = follow_sets[non_terminal]
                for symbol in follow_set_A:
                    parse_table[(non_terminal, symbol)] = production

    return parse_table


# Construct Parse Table
parse_table = construct_parse_table(example_cfg_with_recursion, first_sets, follow_sets)

# Print Parse Table
print("\nParse Table:")
non_terminals = sorted(example_cfg_with_recursion.keys())
terminals = set()

for productions in example_cfg_with_recursion.values():
    for production in productions:
        terminals.update(production)

terminals.difference_update(non_terminals)
terminals.discard('#')  # Remove 'epsilon' from terminals
all_symbols = non_terminals + list(terminals) + ['$']

print("\t" +"\t"+"\t".join(terminals))

for non_terminal in non_terminals:
    row = [non_terminal]
    for symbol in all_symbols[1:]:
        row.append("".join(str(step) for step in parse_table.get((non_terminal, symbol), [''])))
    print("\t".join(row))



Parse Table:
		a	c	b
A		aA			
S		Ab	c		
